# What happens:

- last_input is done

## Import 

In [1]:
import os
import sys
import pandas as pd
import numpy as np
from pandas import DataFrame
import sqlalchemy
from sqlalchemy import create_engine
import getpass
import datetime
import win32com.client as win32
import calendar
from openpyxl import Workbook, load_workbook
from IPython.lib.display import Audio
from openpyxl.utils.dataframe import dataframe_to_rows

### Select the folder where you wanna do the work
### Also ask for the Reporting Date

In [3]:
directory = "C:/Users/WZHARBC/Documents/Python Monthly Tasks/Exports/Retail Data Process"
os.chdir(directory)

rd = str(input("Please enter the RD (YYYYMMDD): "))

ws = "0194"

Please enter the RD (YYYYMMDD): 20211031


### Get cno password

In [25]:
# Getting user from env
cno_user = "CNO_" + getpass.getuser()

# Prompting for pwd
print("Please, enter the password for your user: ", cno_user)
cno_pwd = getpass.getpass()
if(cno_pwd == ""):
    print("Error - password entered as blank")
else:
    print("Password entered")

Please, enter the password for your user:  CNO_WZHARBC
········
Password entered


### Make connection with Oracle DB

In [26]:
# Ray Connection
logon_string_RAY = str("oracle+cx_oracle://" + cno_user + ":" + cno_pwd + "@localhost:41521/PN001")
connection_RAY = create_engine(logon_string_RAY)
connection_RAY.execute("call pack_context.context_open(to_date('{reporting_date}','YYYYMMDD'), {partition})".format(
                        reporting_date = rd, partition = ws))

### Check if data is delivered

In [27]:
%%time
d1 = pd.read_sql_query("""select * from v_rzb_cdr_retail where rownum < 50""", connection_RAY)
if( len(d1) > 0):
    print("Ready to go")
else:
    print("Retail data is not yet delivered")

Ready to go
Wall time: 374 ms


### Get months

In [28]:
rd1 = 0
rd2 = 0
rd3 = 0
def get_months():
    global rd, rd1, rd2, rd3
    #seperate to y, m and d
    year = rd[0:4]
    month = rd[4:6]
    day = rd[6:8]
    rd1 = rd
    if(month == "01") | (month == "02"):
        if(month == "01"):
            #update month
            year = str(int(year) - 1)
            month = "12"
            
            rd2 = str(year) + str(int(month)) + str(calendar.monthrange(int(year), int(month))[1])
            rd3 = str(year) + str(int(month)-1) + str(calendar.monthrange(int(year), int(month)-1)[1])
        else:
            rd2 = str(year) + "0" + str(int(month)-1) + str(calendar.monthrange(int(year), int(month)-1)[1])
            #update month
            year = str(int(year) - 1)
            month = "12"
            
            rd3 = str(year) + str(int(month)) + str(calendar.monthrange(int(year), int(month))[1])
    else:
        if(int(month) <11 ):
            rd2 = str(year) + "0" + str(int(month)-1) + str(calendar.monthrange(int(year), int(month)-1)[1])
            rd3 = str(year) + "0" + str(int(month)-2) + str(calendar.monthrange(int(year), int(month)-2)[1])
        elif (int(month) == 11):
            rd2 = str(year) + str(int(month)-1) + str(calendar.monthrange(int(year), int(month)-1)[1])
            rd3 = str(year) + "0" + str(int(month)-2) + str(calendar.monthrange(int(year), int(month)-2)[1])
        else:
            rd2 = str(year) + str(int(month)-1) + str(calendar.monthrange(int(year), int(month)-1)[1])
            rd3 = str(year) + str(int(month)-2) + str(calendar.monthrange(int(year), int(month)-2)[1])  
    
get_months()

### Open excel file

In [6]:
# determining the name of the file
file_name = 'Retail_monthly_check_' + calendar.month_name[int(rd[4:6])] + '_' + rd[0:4] + '.xlsx'

retail = load_workbook("test.xlsm")
if not retail.views:
    retail.views.append(openpyxl.workbook.views.BookView())


input_last = retail['INPUT_LAST']
# cell_range = input_last['A2':'AD20000']

# #clear sheet
# for row in cell_range:
#     for cell in row:
#         cell.value = None  
        
input_current = retail['INPUT_CURRENT']
# cell_range = input_current['A2':'AD20000']

# #clear sheet
# for row in cell_range:
#     for cell in row:
#         cell.value = None 

ValueError: year 3396672 is out of range

### Go to all contexts

In [58]:
connection_RAY.execute("call pack_context.context_disable()");

### Select last input

In [59]:
d1 = pd.read_sql_query(""" select reporting_date, type, booking_company_new,counterparty_new_category,bs_type, contract_type, currency_pool, ltv, internal_rating, sum(ead_pre_ccf_agg) as ead_pre_ccf_agg, sum(defaulted) as defaulted, sum(rwa) as rwa, sum(rwa_risk) as rwa_risk,  sum(ead_pro) as provision, sum(rating_sum) as rating, sum(ltv_sum) as ltv, sum(currency_pool_sum) as currency_pool, sum(contract_type_sum) as contract_type, cut_date,delinquency,sum(amount_in_default) as amount_in_default, sum(amount_in_default_lcy) as amount_in_default_lcy, sum(ead_pre_ccf_detail) as ead_pre_ccf_detail,   
sum(ead_pre_ccf_agg) as ead_pre_ccf_agg_fx, sum(rwa) as rwa_fx, sum(rwa_risk) as rwa_risk_fx, sum(defaulted) as defaulted_fx,  sum(ead_pro) as provision_fx , sum(ead_on) as ead_on, sum(ead_off) as ead_off
from(with retail as (select * from rzb_cdr_retail where partition_key = '""" + rd2 + """0000' and import_source = 'RETAIL_RRM')--,   
--ccy_current as (select ccy_code, exch_rate from currency where partition_key = '""" + rd2 + """0000'),   
--ccy_new as (select ccy_code, exch_rate from currency where partition_key = '201505310000'),   
--ccy_old as (select ccy_code, exch_rate from currency where partition_key = '201504300000' )   
select  (case booking_company   
  when 'RFNLRO' then 'RBRO'  
  when 'RFNLBG' then 'RBBG'  
  when 'SPRO01' then 'RBRO' 
  when 'SPBG01' then 'RBBG'  
  when 'ROOFRU' then 'RBRU'  
  when 'RFNLRS' then 'RBRS' 
  else booking_company end) as booking_company_new, 
  (case  
  when booking_company like 'RL%' then 'Leasing' 
  when booking_company like 'RA%' then 'Leasing' 
  when booking_company = 'TLSK' then 'Leasing' 
  when booking_company = 'HULE' then 'Leasing' 
  else 'Bank' end) as Type, 
  (case  
  when delinquency in ('> 181 dpd new','91 - 120 dpd','151 - 180 dpd','> 181 dpd','121 - 150 dpd')  
   then ead_pre_ccf_detail
  else 0 end) as defaulted, 
case when internal_rating is not null   
then ead_pre_ccf_detail   
else 0 end as rating_sum,   
case when ltv is not null   
then ead_pre_ccf_detail   
else 0 end as ltv_sum,   
case when currency_pool is not null   
then ead_pre_ccf_detail   
else 0 end as currency_pool_sum,   
case when (contract_type is not null and contract_type != 'NO MAP')   
then ead_pre_ccf_agg   
else 0 end as contract_type_sum,       
  bs_type,  
  reporting_Date,  
  counterparty_new_category,  
  ead_pre_ccf_agg, 
  --    ead_pre_ccf_agg*n.exch_rate/o.exch_rate as ead_pre_ccf_agg_fx,   
  rwa, 
  --    rwa*n.exch_rate/o.exch_rate as rwa_fx,   
  rwa_risk, 
  --    rwa_risk*n.exch_rate/o.exch_rate as rwa_risk_fx,   
  ead_pro, 
ltv,   
internal_rating,   
currency_pool,   
contract_type,   
cut_date,   
delinquency,   
ead_pre_ccf_detail,   
amount_in_default,   
amount_in_default_lcy,
ead_on,
ead_off--,   
--(case   
--when to_date(substr(partition_key, 1,8),'YYYYMMDD') = cut_date then c.exch_rate    
 --else n.exch_rate end   
--) as exch_new,   
--(case   
--when to_date(substr(partition_key, 1,8),'YYYYMMDD') = cut_date then n.exch_rate    
 --else o.exch_rate end   
--) exch_old   
  from retail r  
 --left join ccy_new n on r.currency = n.ccy_code   
 --left join ccy_old o on r.currency = o.ccy_code   
--left join ccy_current c on r.currency = c.ccy_code   
)   
where counterparty_new_category in ('MICRO', 'PI', 'SE') 
group by booking_company_new, type, reporting_Date, counterparty_new_category, bs_type, contract_type, currency_pool, ltv, internal_rating, cut_date,delinquency """, connection_RAY)

In [60]:
for i, row in enumerate(d1.itertuples()):
    for j, val in enumerate(row[1:]):
        input_last.cell(row=2+i, column=1+j).value = val

### Select current input

In [61]:
d1 = pd.read_sql_query("""select reporting_date, type, booking_company_new,counterparty_new_category,bs_type, contract_type, currency_pool, ltv, internal_rating, sum(ead_pre_ccf_agg) as ead_pre_ccf_agg, sum(defaulted) as defaulted, sum(rwa) as rwa, sum(rwa_risk) as rwa_risk,  sum(ead_pro) as provision, sum(rating_sum) as rating, sum(ltv_sum) as ltv, sum(currency_pool_sum) as currency_pool, sum(contract_type_sum) as contract_type, cut_date,delinquency,sum(amount_in_default) as amount_in_default, sum(amount_in_default_lcy) as amount_in_default_lcy, sum(ead_pre_ccf_detail) as ead_pre_ccf_detail,   
sum(ead_pre_ccf_agg*exch_new/exch_old) as ead_pre_ccf_agg_fx, sum(rwa*exch_new/exch_old) as rwa_fx, sum(rwa_risk*exch_new/exch_old) as rwa_risk_fx, sum(defaulted*exch_new/exch_old) as defaulted_fx,  sum(ead_pro*exch_new/exch_old) as provision_fx,sum(ead_on) as ead_on, sum(ead_off) as ead_off   
from(with retail as (select * from rzb_cdr_retail where partition_key = '"""+rd1+"""0000'),   
ccy_current as (select ccy_code, exch_rate from currency where partition_key = '"""+rd1+"""0000'),   
ccy_new as (select ccy_code, exch_rate from currency where partition_key = '"""+rd2+"""0000'),   
ccy_old as (select ccy_code, exch_rate from currency where partition_key = '"""+rd3+"""0000' )   
select  (case booking_company   
  when 'RFNLRO' then 'RBRO'  
  when 'RFNLBG' then 'RBBG'  
  when 'SPRO01' then 'RBRO' 
  when 'SPBG01' then 'RBBG'  
  when 'ROOFRU' then 'RBRU'  
  when 'RFNLRS' then 'RBRS' 
  else booking_company end) as booking_company_new, 
  (case  
  when booking_company like 'RL%' then 'Leasing' 
  when booking_company like 'RA%' then 'Leasing' 
  when booking_company = 'TLSK' then 'Leasing' 
  when booking_company = 'HULE' then 'Leasing' 
  else 'Bank' end) as Type, 
  (case  
  when delinquency in ('> 181 dpd new','91 - 120 dpd','151 - 180 dpd','> 181 dpd','121 - 150 dpd')  
   then ead_pre_ccf_detail
  else 0 end) as defaulted, 
case when internal_rating is not null   
then ead_pre_ccf_detail   
else 0 end as rating_sum,   
case when ltv is not null   
then ead_pre_ccf_detail   
else 0 end as ltv_sum,   
case when currency_pool is not null   
then ead_pre_ccf_detail   
else 0 end as currency_pool_sum,   
case when (contract_type is not null and contract_type != 'NO MAP')   
then ead_pre_ccf_agg   
else 0 end as contract_type_sum,       
  bs_type,  
  reporting_Date,  
  counterparty_new_category,  
  ead_pre_ccf_agg, 
  --    ead_pre_ccf_agg*n.exch_rate/o.exch_rate as ead_pre_ccf_agg_fx,   
  rwa, 
  --    rwa*n.exch_rate/o.exch_rate as rwa_fx,   
  rwa_risk, 
  --    rwa_risk*n.exch_rate/o.exch_rate as rwa_risk_fx,   
  ead_pro, 
ltv,   
internal_rating,   
currency_pool,   
contract_type,   
cut_date,   
delinquency,   
ead_pre_ccf_detail,   
amount_in_default,   
amount_in_default_lcy,  
ead_on,
ead_off, 
(case   
when to_date(substr(partition_key, 1,8),'YYYYMMDD') = cut_date then c.exch_rate    
else n.exch_rate end   
) as exch_new,   
(case   
when to_date(substr(partition_key, 1,8),'YYYYMMDD') = cut_date then n.exch_rate    
else o.exch_rate end   
) exch_old   
  from retail r  
left join ccy_new n on r.currency = n.ccy_code   
left join ccy_old o on r.currency = o.ccy_code   
left join ccy_current c on r.currency = c.ccy_code   
)   
where counterparty_new_category in ('MICRO', 'PI', 'SE')   
group by booking_company_new, type, reporting_Date, counterparty_new_category, bs_type, contract_type, currency_pool, ltv, internal_rating, cut_date,delinquency
""", connection_RAY)

In [62]:
for i, row in enumerate(d1.itertuples()):
    for j, val in enumerate(row[1:]):
        input_current.cell(row=2+i, column=1+j).value = val

In [12]:
rows = dataframe_to_rows(d1)
for r_idx, row in enumerate(rows, 1):
    for c_idx, value in enumerate(row, 1):
         input_last.cell(row=r_idx, column=c_idx, value=value)
            
input_last.delete_cols(1)

### Select Units from both WS

In [8]:
%%time
d2 = pd.read_sql_query("""select distinct booking_company
from rzb_Cdr
where ( attribute_5 like '%KLI%' or attribute_5 like '%WTC%' ) and booking_company not in ('RKAG','RFACTOR','RBRUFIN','RALT','RBSPK','RBSPKHR','RBSPKMS','RBSPKRO','RLBY1','RSTS','RWBB','ALEASS','AREALE')
""", connection_RAY)

Wall time: 22.9 s


In [9]:
for i, row in d2.iterrows():
    first_range = i
    ws.cell(row=i+1, column=1).value = d2.booking_company[i]

In [10]:
connection_RAY.execute("call pack_context.context_open(to_date('{reporting_date}','YYYYMMDD'), {partition})".format(
                        reporting_date = rd, partition = ws_cons))

In [11]:
%%time
d3 = pd.read_sql_query("""select distinct booking_company
from rzb_Cdr
where ( attribute_5 like '%KLI%' or attribute_5 like '%WTC%' ) and booking_company not in ('RKAG','RFACTOR','RBRUFIN','RALT','RBSPK','RBSPKHR','RBSPKMS','RBSPKRO','RLBY1','RSTS','RWBB','ALEASS','AREALE')
""", connection_RAY)

Wall time: 38.3 s


In [12]:
for i, row in d3.iterrows():
    ws.cell(row=1+i+first_range, column=1).value = d3.booking_company[i]

In [63]:
retail.save(file_name)

In [ ]:
framerate = 4410
play_time_seconds = 3

t = np.linspace(0, play_time_seconds, framerate*play_time_seconds)
audio_data = np.sin(2*np.pi*300*t) + np.sin(2*np.pi*240*t)
Audio(audio_data, rate=framerate, autoplay=True)